# Importação das bibliotecas e do dataset

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TRT05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Carregando embedding da USP

In [2]:
usp_word2vec = KeyedVectors.load_word2vec_format('cbow_s100.txt')

In [3]:
print(len(usp_word2vec['pescoço']))

100


# Aplicações

In [4]:
print(usp_word2vec['homem'])

[ 3.58052e-01 -2.33786e-01 -2.08213e-01 -5.26642e-01  3.37073e-01
  2.78175e-01  4.57310e-02 -9.48120e-02  1.61410e-01 -3.39540e-02
 -9.34600e-03 -1.77987e-01 -4.97860e-01 -2.77990e-02  1.30160e-02
  9.09000e-04 -1.83411e-01  1.71000e-04  2.08070e-02  1.46358e-01
  2.08362e-01  2.83546e-01 -2.48896e-01  3.97750e-01 -9.65050e-02
  8.90710e-02  1.42334e-01 -8.21470e-02  1.88183e-01  1.94576e-01
  1.80388e-01 -2.16986e-01  6.78420e-02  1.38773e-01  2.81078e-01
  2.83760e-02  3.40319e-01 -1.50252e-01  1.02771e-01  5.24210e-02
 -1.89300e-03  1.61379e-01  1.30034e-01 -1.82657e-01  1.50900e-01
 -1.31224e-01 -1.02074e-01 -2.51900e-03 -1.40393e-01 -1.37729e-01
 -5.61260e-02 -3.66584e-01 -5.09300e-03 -2.44375e-01 -1.96195e-01
 -3.53230e-02  3.91601e-01 -8.46560e-02  3.98038e-01 -2.97209e-01
 -1.50558e-01  7.86980e-02  2.06097e-01 -1.35907e-01  3.12400e-02
  3.78501e-01  3.04121e-01 -1.01285e-01  8.54210e-02 -1.97107e-01
 -4.00851e-01 -3.65820e-02 -1.63780e-01 -3.82440e-02 -3.63000e-02
 -2.56132e

In [5]:
def dist_euclidiana(u, v):
  return np.linalg.norm(u - v)

In [6]:
homem = usp_word2vec['homem']
rapaz = usp_word2vec['rapaz']
menino = usp_word2vec['menino']
mulher = usp_word2vec['mulher']
menina = usp_word2vec['menina']

print(dist_euclidiana(homem, rapaz))
print(dist_euclidiana(homem, menino))
print(dist_euclidiana(homem, mulher))
print(dist_euclidiana(homem, menina))

0.8654262
1.3192099
2.7291093
2.6769214


In [7]:
def similaridade_cossenos(u, v):
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [8]:
print(similaridade_cossenos(homem, rapaz))
print(similaridade_cossenos(homem, menino))
print(similaridade_cossenos(homem, mulher))
print(similaridade_cossenos(homem, menina))

0.9280618
0.8123638
0.22989851
0.28614414


In [9]:
def analogia(x1, x2, y1):
  y2 = usp_word2vec.most_similar(positive = [y1, x2], negative = [x1])
  return y2

In [10]:
filho = usp_word2vec['filho']

analogia(homem, mulher, filho)

[('filha', 0.8104748129844666),
 ('sobrinha', 0.7517995834350586),
 ('esposa', 0.7465153336524963),
 ('irmã', 0.7406715750694275),
 ('neta', 0.732083261013031),
 ('namorada', 0.7275243401527405),
 ('mulher', 0.7230202555656433),
 ('enteada', 0.7192704677581787),
 ('meia-irmã', 0.7150097489356995),
 ('governanta', 0.7126821875572205)]

In [11]:
cachorro = usp_word2vec['cachorro']
cadela = usp_word2vec['cadela']
gato = usp_word2vec['gato']

analogia(cachorro, cadela, gato)

[('cadela', 0.8309482932090759),
 ('cachorrinha', 0.7623376250267029),
 ('bruxa', 0.7476592659950256),
 ('mulher', 0.7465581297874451),
 ('girafa', 0.7449819445610046),
 ('rapariga', 0.7252045273780823),
 ('mula', 0.7197816371917725),
 ('égua', 0.7185747623443604),
 ('criatura', 0.7105768918991089),
 ('porquinha', 0.7102939486503601)]

In [12]:
medico = usp_word2vec['médico']

analogia(homem, medico, mulher)

[('enfermeira', 0.7371505498886108),
 ('mulher', 0.6708241105079651),
 ('funcionária', 0.6609692573547363),
 ('psicóloga', 0.6459536552429199),
 ('videmanette', 0.6337722539901733),
 ('ecografia', 0.6319240927696228),
 ('bibliotecária', 0.6272015571594238),
 ('prostituta', 0.6178702712059021),
 ('aluna', 0.615606963634491),
 ('creche', 0.6106577515602112)]

In [13]:
medica = usp_word2vec['médica']

analogia(mulher, medica, homem)

[('detento', 0.5940698981285095),
 ('reploíde', 0.5839069485664368),
 ('anti-depressivo', 0.5656048059463501),
 ('não-acordo', 0.5616338849067688),
 ('homem', 0.5586972832679749),
 ('texto.a', 0.5491770505905151),
 ('taxista', 0.545571506023407),
 ('lenã§ol', 0.5454291701316833),
 ('serviçode', 0.5445601344108582),
 ('tamponando', 0.5398924946784973)]

In [14]:
programador = usp_word2vec['programador']

analogia(homem, programador, mulher)

[('máquina', 0.6583220958709717),
 ('ex-namorada', 0.6497247815132141),
 ('mulher', 0.6463732719421387),
 ('mergulhadora', 0.6403951644897461),
 ('sugestã£o', 0.6394364833831787),
 ('parceira', 0.6380342841148376),
 ('duplicata', 0.6358045339584351),
 ('imagem-tipo', 0.6309173703193665),
 ('impressora', 0.6299152970314026),
 ('enfermeira', 0.6262940764427185)]

In [15]:
programadora = usp_word2vec['programadora']

analogia(mulher, programadora, homem)

[('programador', 0.6169764995574951),
 ('conceito', 0.601061224937439),
 ('agorafóbico', 0.5994553565979004),
 ('desenvolvedor', 0.595237135887146),
 ('gerenciador', 0.5938831567764282),
 ('decodificador', 0.583011269569397),
 ('exploit', 0.5821406245231628),
 ('instalador', 0.5817169547080994),
 ('videojogo', 0.5804249048233032),
 ('portmanteau', 0.5788525938987732)]

In [16]:
rei = usp_word2vec['rei']

analogia(homem, mulher, rei)

[('esposa', 0.692970871925354),
 ('filha', 0.6821932196617126),
 ('concubina', 0.6798956394195557),
 ('governanta', 0.67839115858078),
 ('sobrinha', 0.6778334379196167),
 ('madrasta', 0.6712793111801147),
 ('dama-de-companhia', 0.6649885773658752),
 ('múmia', 0.6630989909172058),
 ('bisneta', 0.6615864038467407),
 ('benção', 0.6589256525039673)]

In [17]:
rainha = usp_word2vec['rainha']

analogia(mulher, homem, rainha)

[('homem', 0.6283648610115051),
 ('rei', 0.5936132073402405),
 ('moço', 0.57784503698349),
 ('herói', 0.56794673204422),
 ('prelado', 0.5664745569229126),
 ('rapaz', 0.5660328269004822),
 ('frêmito', 0.5653231143951416),
 ('panegírico', 0.5649533867835999),
 ('morgadio', 0.5629957914352417),
 ('pescador', 0.5626760125160217)]

In [18]:
cantar = usp_word2vec['cantar']
cantando = usp_word2vec['cantando']
dancar = usp_word2vec['dançar']

analogia(cantar, cantando, dancar)

[('cantando', 0.8179231286048889),
 ('dançando', 0.7392323017120361),
 ('dançar', 0.6498607397079468),
 ('tocando', 0.6495398283004761),
 ('aplaudindo', 0.5950785279273987),
 ('cantarolando', 0.5816349983215332),
 ('vibrando', 0.5793366432189941),
 ('assobiando', 0.5731126070022583),
 ('pescando', 0.561478316783905),
 ('compondo', 0.5515736937522888)]